In [ ]:
pip install nltk

In [ ]:
pip install newspaper3k 

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 194kB 45.8MB/s 
     |████████████████████████████████| 7.4MB 34.3MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=a27123883b417789e272c14d17e1790b00730ab991078c25e1cc9e87464a49c9
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=99cdf9f8d778869acc2e57f19568b3851442c783153a5f8d8f938480e8defdbe
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=5db14a3a2680174b514d3214568f52191c49f057f524f9ac2fdad4fc3b8d0bf6
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da2

In [ ]:
#This bot goes to a website, scrapes information off of the website, let user write query to chatbot and chatbot will find most similar sentence to user query on the website and use that to respond to them.

In [ ]:
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet=True)

True

In [ ]:
article = Article('https://industrytoday.com/the-rise-of-remanufacturing/')

In [ ]:
article.download()
article.parse()
article.nlp()
corpus = article.text
print(corpus)

February 12, 2019

Architecture company Rau designed Brummen Town Hall in the Netherlands to last just 20 years. Due to shifting municipality borders, the firm opted for reusable materials that will be dismantled and returned to manufacturers at the end of the building’s working life.

90 per cent of the materials used to build Brummen Town Hall are recyclable. At the end of the building’s life, little material will go into landfill — instead it will be repurposed for other projects. Manufacturers too are considering how materials can be reused, one increasingly popular option is remanufacturing. This is the process of disassembling, cleaning and repairing components so that the product can be reassembled for sale at the performance and quality of a new product.

The technique is predominantly used in the aerospace and automotive sectors. Despite having a history of several decades, there remains some confusion about the differences between reconditioned and remanufactured products. Re

In [ ]:
#Tokenization
text = corpus
sent_tokens = nltk.sent_tokenize(text) #converts text into list of sentences
print(sent_tokens)

['February 12, 2019\n\nArchitecture company Rau designed Brummen Town Hall in the Netherlands to last just 20 years.', 'Due to shifting municipality borders, the firm opted for reusable materials that will be dismantled and returned to manufacturers at the end of the building’s working life.', '90 per cent of the materials used to build Brummen Town Hall are recyclable.', 'At the end of the building’s life, little material will go into landfill — instead it will be repurposed for other projects.', 'Manufacturers too are considering how materials can be reused, one increasingly popular option is remanufacturing.', 'This is the process of disassembling, cleaning and repairing components so that the product can be reassembled for sale at the performance and quality of a new product.', 'The technique is predominantly used in the aerospace and automotive sectors.', 'Despite having a history of several decades, there remains some confusion about the differences between reconditioned and rema

In [ ]:
#Creates dictionary (key:value) pair to remove punctuation
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
print(string.punctuation) #print punctuations
print(remove_punct_dict)#print dictionary

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [ ]:
#create function to return list of lemmetized lower case words after removing punctuation
def lemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict)) #use this to remove any leftover puctuation
print(lemNormalize(text)) #print tokenizaztion text

['february', '12', '2019', 'architecture', 'company', 'rau', 'designed', 'brummen', 'town', 'hall', 'in', 'the', 'netherlands', 'to', 'last', 'just', '20', 'years', 'due', 'to', 'shifting', 'municipality', 'borders', 'the', 'firm', 'opted', 'for', 'reusable', 'materials', 'that', 'will', 'be', 'dismantled', 'and', 'returned', 'to', 'manufacturers', 'at', 'the', 'end', 'of', 'the', 'building', '’', 's', 'working', 'life', '90', 'per', 'cent', 'of', 'the', 'materials', 'used', 'to', 'build', 'brummen', 'town', 'hall', 'are', 'recyclable', 'at', 'the', 'end', 'of', 'the', 'building', '’', 's', 'life', 'little', 'material', 'will', 'go', 'into', 'landfill', '—', 'instead', 'it', 'will', 'be', 'repurposed', 'for', 'other', 'projects', 'manufacturers', 'too', 'are', 'considering', 'how', 'materials', 'can', 'be', 'reused', 'one', 'increasingly', 'popular', 'option', 'is', 'remanufacturing', 'this', 'is', 'the', 'process', 'of', 'disassembling', 'cleaning', 'and', 'repairing', 'components', '

In [ ]:
#Keyword Input
Greeting_inputs = ["hi","hello","hey","I have a question"]
Greeting_responses = ["hello, how can i help you", "howdy","hi what's up"]

#Function to return random greeting response
def greeting(sentence):
  for word in sentence.split():
    if word.lower in Greeting_inputs:
      return random.choice(Greeting_responses)

In [ ]:
#Generate response
def response(user_response):

  #user_response = "what is environmental volunteering abroad" #query
  user_response = user_response.lower()

  print(user_response)
#set chatbot response to empty string
  robo_response= ''

#appending user response to sentence list
  sent_tokens.append(user_response)
#print sentence list after appending user's response
  print(sent_tokens)
#create TFIDFVectorizer object
  tfidfVec = TfidfVectorizer(tokenizer=lemNormalize,stop_words="english")

#create a matrix of turn freq-inverse document frequency [TFIDF] features
  tfidf = tfidfVec.fit_transform(sent_tokens)

  print(tfidf)
#Get measure of similarity: similarity scores (comparing text scrapped from website to user's question)
  vals = cosine_similarity(tfidf[-1],tfidf)

  print(vals)
#To find sentence most similar to user's query
  idx = vals.argsort()[0][-2] #user'squery stored at -1:best match and so we want -2:next best reponse which is our answer to their question (most simialr score stored at very end)

#reduce dimentionality of vals (vals is a list os lists so we want to make it into just one list)
  flat = vals.flatten()

#sort list in ascending order
  flat.sort()

#get most similar score to user response
  score = flat[-2]

#print simialrity score
  print(score)
#if variable score = 0, no text similar to user response
  if (score ==0):
    robo_response = robo_response + "I don't understand."

  else:
    robo_response = robo_response + sent_tokens[idx]
  sent_tokens.remove(user_response)
#chatbot response
  print (sent_tokens[idx])
  return robo_response

#remove user query from sentence tokens after answering
  #sent_tokens.remove(user_response)

In [ ]:
flag = True
print("Hello, I am chatbot. Let me answer any questions you have on this article! If you want to leave at any point, type STOP.")
while (flag ==True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'STOP'):
    if(greeting(user_response) != (None)):
      print ("Chatbot: " + greeting(user_response))
    else:
      print ("Chatbot: " + response(user_response))
      
  
  elif (user_response == 'STOP'):
    flag = False
    print("Chatbot will catch you later!")


Hello, I am chatbot. Let me answer any questions you have on this article! If you want to leave at any point, type STOP.
stop
stop
['February 12, 2019\n\nArchitecture company Rau designed Brummen Town Hall in the Netherlands to last just 20 years.', 'Due to shifting municipality borders, the firm opted for reusable materials that will be dismantled and returned to manufacturers at the end of the building’s working life.', '90 per cent of the materials used to build Brummen Town Hall are recyclable.', 'At the end of the building’s life, little material will go into landfill — instead it will be repurposed for other projects.', 'Manufacturers too are considering how materials can be reused, one increasingly popular option is remanufacturing.', 'This is the process of disassembling, cleaning and repairing components so that the product can be reassembled for sale at the performance and quality of a new product.', 'The technique is predominantly used in the aerospace and automotive sectors

KeyboardInterrupt: ignored